# 理解并实现ResNet和ResNext：从Microsoft到Facebook[Part 2]
本文我们将在Part1的基础上进一步探索ResNet块的最优化函数。讨论ResNeXt结构以及在PyTorch中的实现。
### 关于这个系列
本系列包含两个部分，分别是：<br>
- 理解和实现ResNet结构
- 理解和实现ResNeXt结构

对于已经理解了part1的读者来说，这篇就很简单了。我将沿用part1的行文思路。<br>
1. 简介[深度残差网络](https://arxiv.org/pdf/1603.05027.pdf)的恒等映射。
2. [ResNeXt](https://arxiv.org/pdf/1611.05431.pdf)结构回顾
3. ResNeXt实例研究
4. ResNeXt的Pytorch实现

## 简介深度残差网络的恒等映射
本文将介绍为什么残差网络中不会出现梯度消失，还将介绍将恒等映射替换为不同函数时短路连接扮演的角色。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*h5ailwcPghw9gplG5I3-Wg.png)
F表示一系列非线性层，f表示一个Relu激活函数。<br>
他们发现当f(y1)和h(x1)都是恒等映射时，信号可以从一个单元直接传递到另一个单元，无论在正向还是反向。另外，当它们都是恒等映射时都会达到最小误差率。我们来看一下每一种情况。<br>
### 1.找到最优h(x{l})函数
![avater](https://cdn-images-1.medium.com/max/800/1*QC0ir-cgVHlOqyPzFwaTaw.png)
![avater](https://cdn-images-1.medium.com/max/800/1*2oxBbNY63fJV5gvxtrzXJg.png)
案例1，Lambda = 0：这将是一个平坦网络。由于w2,w1,w0都是在{-1,1}的数，随着网络数量的增加，梯度发生消失。这很清楚地展示了梯度消失问题。<br>
案例2，Lambda > 1：在这个例子中，反向传播的值将迅速增加导致梯度爆炸。<br>
案例3，Lambda < 1：对于浅层网络这可能不是一个问题。但是对于很深的网络，很大多数层上，权重+lambda将仍然小于1，这种情况跟案例1是一样的。<br>
案例4，Lambda = 1: 这种情况下，每个权重都加上了1，这可以避免像案例2一样乘以一个非常大的数或者像案例1一样乘以一个非常小的数。<br>
本文还介绍了通过引入反向传播和卷积层，网络的性能下降了。下面展示的是5个实验，其中只有第一个达到了最小的误差率。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*Y_FttOgxd9KT0OK2I3db-A.png)
![avater](https://cdn-images-1.medium.com/max/800/1*pIl4qZ509HHpj61rCVbiBw.png)

### 2.找到最优的f(y_{1})函数
![avater](https://cdn-images-1.medium.com/max/800/1*lFTw8htrpu5vgiqJpxXMmQ.png)
上面5种结构是在ResNet-110以及ResNet-164的基础上做的研究。在所有的结构中，预激活的情况表现最好。所以一个简单的加法和恒等映射比Relu函数要重要。残差层加入Relu和BN层有助于网络更加快速地收敛，并且获得更少的测试误差，从而国不容易过拟合。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*FbHF_BMZehsmGkg7rVRMOQ.png)

### 结论
所以恒等短路连接（案例1）以及恒等加后激活对于信息的快速传递非常必要。消融实验跟上面讨论的结果是一致的。

## ResNeXt结构概述
ResNeXt获得了ILSVRC2016分类挑战的第二名，在Coco检测数据集以及ImageNet-5k数据集上相较于原始ResNet也获得了性能提升。<br>
论文引入了一个新的概念“基数”。论文简单的解释了这个概念并且将其用在了ResNet中，并且做了一些消融研究。<br>
论文尝试描述Inception的复杂度以及为什么ResNeXt是如此的简单。这里我不再赘述，因为需要读者理解Inception网络。我将只讨论结构。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*mdiQTfovOXKnqzfj727b9Q.png)
- 上图是对ResNet块以及ResNeXt块的对比
- 遵循分变换集的策略
- ResNeXt块中的路径数叫做基数。上图中是32.
- 所有的路径都是相同的拓扑结构
- 不同于很高的深度和宽度，较高的基数有助于减小验证误差。
- ResNeXt尝试嵌入更多的子空间
- 这两种结构有不同的宽度。ResNet中的Layer-1有一个宽度为64的卷积层，然而ResNext的layer-1有32个不同的卷积层，每个的宽度是4(32\*4)。尽管ResNext的宽度看起来更宽，但是参数却是一样的（大约7万）（ResNet 256\*64+3\*3\*64\*64+64\*26）(ResNeXt C\*(256\*d+3\*3\*d\*d+d\*256),这里C=32,d=4)

### 下面是ResNet和ResNeXt的区别
![avater](https://cdn-images-1.medium.com/max/800/1*K03QPmjwWQgjNusXRK4Myw.png)
一个resnext_32\*4d 表示有4个瓶颈的网络，如上图所示。每个层都有32个基数。我们将会看到resnext_32\*4d和resnext_64\*4d的pytorch实现。<br>
### 研究
**基数 vs 宽度：**当C从1增加到32时，我们可以清楚地观察到误差下降了1%。因此，增加C而减小宽度可以提升模型的性能。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*VmsRr6jis0Sg7asAfAT1LQ.png)
**增加基数 vs 更深/更宽：**通过三个案例来看。<br>
1) 从101层增加到200层
2) 加宽瓶颈3层
3) 将基数C翻倍
结果显示增加C会得到更好的结果。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*t4CD5LCb-4TtFgXry-Sndg.png)

## 结论
不同的ResNeXt结构集成后会得到3.03%的top5误差率，这个成绩赢得了ILSVRC的第二名。<br>
该架构相对于Inception要简单一些。<br>

## Pytorch实现
ResNeXt目前还没有得到Pytorch的官方支持。Cadene[实现](https://github.com/Cadene/pretrained-models.pytorch)了一个并且有一个可用的预训练的权重。<br>
我在另一篇[博文](https://medium.com/@14prakash/almost-any-image-classification-problem-using-pytorch-i-am-in-love-with-pytorch-26c7aa979ec4)中介绍了如何使用这个知识库。你可以找到ResNeXt的实现。ResNeXt-32\*4d以及ResNeXt-64\*4d的预训练版实现都可以找到。